In [1]:
from docplex.mp.model import Model
import pandas as pd
from pandas import DataFrame
import datetime
import xlrd
import matplotlib.pyplot as plt 
import numpy as np
#loading packages that are needed

In [2]:
name = 'November_2025_master_code/Meeting_Patterns_for_GitHub_small'#'Meeting_Patterns_July_2025' #'Meeting_Patterns_small_Jan_2025' #'Meeting_Patterns_Fall_2025' #'Meeting_Patterns_Medium_Nov_2024'
class_file_name = name + ".xlsx"


In [3]:
class_xls_file = pd.ExcelFile(class_file_name)
df_time_group_info = class_xls_file.parse('Time_Group_Info')
df_time_opts_og = class_xls_file.parse('Time_Options')

In [4]:
df_time_opts_og

,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes
0,1,MWF,01:00:00,01:50:00,L,NaN
1,1,R,01:00:00,02:50:00,L,(Math 347)
2,2,MWRF,08:00:00,08:50:00,H,NaN
3,3,MWRF,09:00:00,09:50:00,H,NaN
4,4,MWRF,01:00:00,01:50:00,H,NaN
5,5,MWRF,02:00:00,02:50:00,H,NaN
6,6,MWRF,03:00:00,03:50:00,H,NaN


In [5]:
start_time_minutes_past_midnight_dict = {}
end_time_minutes_past_midnight_dict = {}
for start_time in df_time_opts_og['Start_Time'].unique():
    if not issubclass(type(start_time), float):
        start_time_min = 0
        end_time = 0
        if start_time >= datetime.time(8,0,0):
            start_time_min = (start_time.hour)*60+start_time.minute
        else:
            start_time_min = (start_time.hour+12)*60 + start_time.minute
        end_time_per_start = df_time_opts_og.loc[df_time_opts_og['Start_Time']==start_time]['End_Time'].unique()
        for end_time in end_time_per_start:
            if end_time > datetime.time(8,0,0):
                end_time_min = (end_time.hour*60)+end_time.minute
            else:
                end_time_min = (end_time.hour+12)*60 + end_time.minute
            start_time_minutes_past_midnight_dict[start_time] = start_time_min
            end_time_minutes_past_midnight_dict[end_time] = end_time_min
# for start_time in df_time_opts_og.itertuples():
#     if issubclass(type(start_time[3]), float):
#         start_time_minutes_past_midnight.append(0)
#         end_time_minutes_past_midnight.append(0)
#     else:
#         if start_time[3] < datetime.time(8,0,0):
#             start_time_min = (start_time[3].hour+12)*60 + start_time[3].minute
#         if start_time[4] < datetime.time(8,0,0):
#              end_time_min = (start_time[4].hour+12)*60 + start_time[4].minute
#         else:
#             start_time_min = (start_time[3].hour)*60+start_time[3].minute
#             end_time_min = (start_time[4].hour*60)+start_time[4].minute
#         start_time_minutes_past_midnight.append(start_time_min)
#         end_time_minutes_past_midnight.append(end_time_min)
df_time_opts_og['start_time_past_midnight'] = df_time_opts_og['Start_Time'].map(start_time_minutes_past_midnight_dict)
df_time_opts_og['end_time_past_midnight'] = df_time_opts_og['End_Time'].map(end_time_minutes_past_midnight_dict)
#start_time_minutes_past_midnight
df_time_opts_og

,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes,start_time_past_midnight,end_time_past_midnight
0,1,MWF,01:00:00,01:50:00,L,NaN,780,830
1,1,R,01:00:00,02:50:00,L,(Math 347),780,890
2,2,MWRF,08:00:00,08:50:00,H,NaN,480,530
3,3,MWRF,09:00:00,09:50:00,H,NaN,540,590
4,4,MWRF,01:00:00,01:50:00,H,NaN,780,830
5,5,MWRF,02:00:00,02:50:00,H,NaN,840,890
6,6,MWRF,03:00:00,03:50:00,H,NaN,900,950


In [6]:
df_time_opts_cleaned = df_time_opts_og.drop(columns=['Notes'])
df_time_opts = df_time_opts_cleaned.dropna()
df_time_opts

,Time_Index,Days,Start_Time,End_Time,Time_Group,start_time_past_midnight,end_time_past_midnight
0,1,MWF,01:00:00,01:50:00,L,780,830
1,1,R,01:00:00,02:50:00,L,780,890
2,2,MWRF,08:00:00,08:50:00,H,480,530
3,3,MWRF,09:00:00,09:50:00,H,540,590
4,4,MWRF,01:00:00,01:50:00,H,780,830
5,5,MWRF,02:00:00,02:50:00,H,840,890
6,6,MWRF,03:00:00,03:50:00,H,900,950


In [7]:


#### This was overwriting the lecture overlapping times with the lab overlapping times, so I fixed it to combine the two

# for idx in range(df_time_opts.shape[0]):
#     # getting specific time index
#     rows = df_time_opts.iloc[idx] 
    
#     # overlapping times 
#     overlap_time_by_start = df_time_opts.loc[rows['start_time_past_midnight'] <=df_time_opts['start_time_past_midnight']].loc[df_time_opts['start_time_past_midnight']<=rows['end_time_past_midnight']]
#     overlap_time_by_end = df_time_opts.loc[rows['start_time_past_midnight'] <=df_time_opts['end_time_past_midnight']].loc[df_time_opts['end_time_past_midnight']<=rows['end_time_past_midnight']]
#     overlap_times = pd.concat([overlap_time_by_start,overlap_time_by_end])
#     overlap_times.drop_duplicates(inplace=True)
#     # 14 shows up as an overlapping time for 1 here
#     # 1 shows up as an overlapping time for 14 here
#     #print(overlap_times)
    
#     overlap_time_and_day_list = []
    
#     for days_groups in overlap_times['Days']:
#             for day in rows['Days']:
#                 overlap_time_and_day = overlap_times.loc[overlap_times['Days']==days_groups]
#                 #print(overlap_time_and_day)
#                 overlap_time_and_day_list += list(overlap_time_and_day['Time_Index'].values)
#                 overlap_times.drop(index = overlap_time_and_day.index, inplace = True)
#     overlapping_times[rows['Time_Index']] = sorted(overlap_time_and_day_list)



overlapping_times = {}

for idx in range(df_time_opts.shape[0]):
    rows = df_time_opts.iloc[idx] 

    overlap_time_by_start = df_time_opts.loc[rows['start_time_past_midnight'] <=df_time_opts['start_time_past_midnight']].loc[df_time_opts['start_time_past_midnight']<=rows['end_time_past_midnight']]
    overlap_time_by_end = df_time_opts.loc[rows['start_time_past_midnight'] <=df_time_opts['end_time_past_midnight']].loc[df_time_opts['end_time_past_midnight']<=rows['end_time_past_midnight']]
    overlap_times = pd.concat([overlap_time_by_start,overlap_time_by_end])
    overlap_times.drop_duplicates(inplace=True)

    overlap_time_and_day_list = []

    for days_groups in overlap_times['Days']:
        for day in rows['Days']:
            if day in days_groups:
                overlap_time_and_day = overlap_times.loc[overlap_times['Days']==days_groups]
                overlap_time_and_day_list += list(overlap_time_and_day['Time_Index'].values)
                overlap_times.drop(index = overlap_time_and_day.index, inplace = True)
                
    # I added these checks to account for the overwriting and removal of duplicates if both lecture and lab overlap
    if len(df_time_opts["Time_Index"] == rows["Time_Index"]) == 1:        
        overlapping_times[rows['Time_Index']] = list(set(overlap_time_and_day_list))
    elif rows['Time_Index'] not in overlapping_times.keys():
        overlapping_times[rows['Time_Index']] = list(set(overlap_time_and_day_list))
    else:
        overlapping_times[rows["Time_Index"]] = list(set(list(overlapping_times[rows["Time_Index"]]) + list(sorted(overlap_time_and_day_list))))
        
    
df_time_opts_og['overlap_times']=df_time_opts['Time_Index'].map(overlapping_times)
df_time_opts_og


,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes,start_time_past_midnight,end_time_past_midnight,overlap_times
0,1,MWF,01:00:00,01:50:00,L,NaN,780,830,"[1, 4, 5]"
1,1,R,01:00:00,02:50:00,L,(Math 347),780,890,"[1, 4, 5]"
2,2,MWRF,08:00:00,08:50:00,H,NaN,480,530,[2]
3,3,MWRF,09:00:00,09:50:00,H,NaN,540,590,[3]
4,4,MWRF,01:00:00,01:50:00,H,NaN,780,830,"[1, 4]"
5,5,MWRF,02:00:00,02:50:00,H,NaN,840,890,"[1, 5]"
6,6,MWRF,03:00:00,03:50:00,H,NaN,900,950,[6]


In [8]:
# df_time_opts_og.loc[df_time_opts_og['Time_Index']==104]

In [9]:
b2b_dict_start = {}
b2b_dict_end = {}
for idx in range(1, df_time_opts_og.shape[0]):
    df_by_row = df_time_opts_og.iloc[idx]
    back_back_df_start = df_time_opts_og.loc[df_by_row['start_time_past_midnight']-15 <= df_time_opts_og['end_time_past_midnight']].loc[df_time_opts_og['end_time_past_midnight']<=df_by_row['start_time_past_midnight']]
    back_back_df_end = df_time_opts_og.loc[df_by_row['end_time_past_midnight'] <= df_time_opts_og['start_time_past_midnight']].loc[df_time_opts_og['start_time_past_midnight']<=df_by_row['end_time_past_midnight']+15]
#     back_back_df = pd.concat([back_back_df_start, back_back_df_end])
    #back_back_df.drop(index = idx, inplace = True)
    back_back_index_list_start = []
    back_back_index_list_end = []
    for ind_day_row in df_by_row['Days']:
        for day_group_b2b_s in back_back_df_start['Days'].unique():
            if ind_day_row in day_group_b2b_s:
                back_back_with_days = back_back_df_start.loc[back_back_df_start['Days']==day_group_b2b_s]
                back_back_index_list_start += list(back_back_with_days['Time_Index'].values)
                back_back_df_start.drop(index = back_back_with_days.index, inplace = True)
        for day_group_b2b_e in back_back_df_end['Days'].unique():
            if ind_day_row in day_group_b2b_e:
                back_back_with_days = back_back_df_end.loc[back_back_df_end['Days']==day_group_b2b_e]
                back_back_index_list_end += list(back_back_with_days['Time_Index'].values)
                back_back_df_end.drop(index = back_back_with_days.index, inplace = True)
    b2b_dict_start[df_by_row['Time_Index']] = sorted(back_back_index_list_start)
    b2b_dict_end[df_by_row['Time_Index']] = sorted(back_back_index_list_end)
df_time_opts_og['back_to_back_start_times']=df_time_opts['Time_Index'].map(b2b_dict_start)
df_time_opts_og['back_to_back_end_times']=df_time_opts['Time_Index'].map(b2b_dict_end)
df_time_opts_og

,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes,start_time_past_midnight,end_time_past_midnight,overlap_times,back_to_back_start_times,back_to_back_end_times
0,1,MWF,01:00:00,01:50:00,L,NaN,780,830,"[1, 4, 5]",[],[6]
1,1,R,01:00:00,02:50:00,L,(Math 347),780,890,"[1, 4, 5]",[],[6]
2,2,MWRF,08:00:00,08:50:00,H,NaN,480,530,[2],[],[3]
3,3,MWRF,09:00:00,09:50:00,H,NaN,540,590,[3],[2],[]
4,4,MWRF,01:00:00,01:50:00,H,NaN,780,830,"[1, 4]",[],[5]
5,5,MWRF,02:00:00,02:50:00,H,NaN,840,890,"[1, 5]","[1, 4]",[6]
6,6,MWRF,03:00:00,03:50:00,H,NaN,900,950,[6],"[1, 5]",[]


In [10]:
# back_to_back_courses = {}
# df_time_opts_with_bb = df_time_opts_og.drop(columns=['Notes'])
# df_time_opts_cleaned_back_to_back = df_time_opts_with_bb.dropna()
# for rows in df_time_opts_cleaned_back_to_back.itertuples():
#     back_to_back_list = []
#     #back_to_back_list.append(rows[1])
#     day_group_lists_non_index = []
#     for day_groups in df_time_opts_cleaned_back_to_back.itertuples():
#         print(rows[7]-15, day_groups[8], rows[7])
#         if rows[8]+15>=day_groups[7]>=rows[8]:
#             day_group_list = []
#             day_group_list_with_their_indices = {}
#             for day in day_groups[2]:
#                 day_group_list.append(day)
#             if len(day_group_list) > 0:
#                 #print(day_group_list)
#                 day_group_list_with_their_indices[day_groups[1]] = day_group_list
#                 #print(day_group_list_with_their_indices)
#                 day_group_lists_non_index.append(day_group_list_with_their_indices)
#                 #print(day_group_lists_non_index)
#         elif rows[7]-15<=day_groups[8]<=rows[7]:
#             #print(day_groups[1])
#             day_group_list = []
#             day_group_list_with_their_indices = {}
#             for day in day_groups[2]:
#                 day_group_list.append(day)
#             if len(day_group_list) > 0:
#                 day_group_list_with_their_indices[day_groups[1]] = day_group_list
#                 day_group_lists_non_index.append(day_group_list_with_their_indices)
#         #print(day_group_list_row_index)
#         for day_groups_in_others in day_group_lists_non_index:
#             for values in day_groups_in_others.values():
#                 for ind_day_in_others in values:
#                     for days_in_key in day_groups_in_others.keys():
#                         if days_in_key not in back_to_back_list:
#                             back_to_back_list.append(days_in_key)
#     back_to_back_courses[rows[1]] = back_to_back_list
    
# df_time_opts_og['back_to_back_times']=df_time_opts['Time_Index'].map(back_to_back_courses)
# df_time_opts_og


In [11]:
df_time_opts_og.to_excel(f"{name}_overlaps.xlsx", sheet_name='time_options', index=False)
# `index=False` added by Dr. Abra 7/30/2024

In [12]:
#df_time_group_info.to_excel("Time_options_and_groups_with_overlaps_labelled_full.xlsx", sheet_name='group_time_opts')